In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode


@tool()
def weather_forecast(location: str):
    """Use this function to get weather forcast information for a given location e.g France."""
    return f"Forecast: {location} is 36 degrees sunny."


tools = [weather_forecast]

tool_node = ToolNode(tools)

In [ ]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = OllamaFunctions(model="llama3.1", format="json", temperature=0, keep_alive="30m")

llm_with_tools = llm.bind_tools(tools)

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessageGraph
from langgraph.prebuilt import tools_condition

memory = MemorySaver()


class Steps:
    CHAT = "chatbot"
    TOOLS = "tools"
    FINAL = "final"


builder = MessageGraph()

builder.set_entry_point(Steps.CHAT)
builder.add_node(Steps.TOOLS, tool_node)
builder.add_node(Steps.CHAT, llm_with_tools)
builder.add_node(Steps.FINAL, llm)
builder.add_edge(Steps.TOOLS, Steps.FINAL)

builder.add_conditional_edges(Steps.CHAT, tools_condition)

graph = builder.compile(checkpointer=memory)

In [ ]:
for s in graph.stream(
    ("human", "what is the weather like in Paris?"),
    {"configurable": {"thread_id": "thread-1"}},
    stream_mode="values",
):
    s[-1].pretty_print()